In [ ]:
%pip install pydicom

In [10]:
import pydicom
import os
import json
import pandas as pd

In [11]:
def scan_dicom_dir(path):
    for root, dirs, files in os.walk(path):
        dicoms = []
        for file in files:
            if not file.endswith('.dcm'):
                continue
            try:
                d = pydicom.dcmread(f"{root}/{file}")
                if "SeriesDescription" in d:
                    sd = d.SeriesDescription
                else:
                    sd = "NA"
                if "ProtocolName" in d:
                    pn = d.ProtocolName
                else:
                    pn = "NA"
                tn = d.TaskName if "TaskName" in d else "NA"
                rt = d.RepetitionTime if "RepetitionTime" in d else "NA"
                et = d.EchoTime if "EchoTime" in d else "NA"
                it = d.InversionTime if "InversionTime" in d else "NA"
                pst = d.PulseSequenceType if "PulseSequenceType" in d else "NA"
                fa = d.FlipAngle if "FlipAngle" in d else "NA"
                m = d.Manufacturer if "Manufacturer" in d else "NA"
                mo = d.ManufacturersModelName if "ManufacturersModelName" in d else "NA"
                tn = d.TaskName if "TaskName" in d else "NA"
                dic = {"file": file,
                        "root": root, 
                    "SeriesDescription" : sd,
                                "TaskName": tn,
                                "ProtocolName" : pn,
                                "RepetitionTime": rt,
                                "EchoTime": et,
                                "InversionTime": it,
                                "PulseSequenceType": pst,
                                "FlipAngle": fa,
                                "Manufacturer": m,
                                "ManufacturersModelName": mo,}
                dicoms.append(dic)
            except:
                print(f"Failed to load {root}/{file}")
        dirs = [dir for dir in dirs if "." not in dir]
        for dir in dirs:
            little_dic = scan_dicom_dir(f"{root}/{dir}")
            dicoms = dicoms + little_dic
        return  dicoms
        


In [33]:
def clean_dicoms(dicoms):
    df = pd.DataFrame(dicoms)

    sub = df.columns.to_list()
    sub.remove('file')
    unique_rows = df.drop_duplicates(subset=sub)
    return unique_rows

In [13]:
path = "/home/rand/github/LoxLM/LoxLM/20170828-25444"
dicoms = scan_dicom_dir(path)


/home/rand/github/LoxLM/LoxLM/src/.venv/lib/python3.10/site-packages/pydicom/dataset.py:594: UserWarning: Invalid value 'TaskName' used with the 'in' operator: must be an element tag as a 2-tuple or int, or an element keyword
  warnings.warn(msg)
/home/rand/github/LoxLM/LoxLM/src/.venv/lib/python3.10/site-packages/pydicom/dataset.py:594: UserWarning: Invalid value 'PulseSequenceType' used with the 'in' operator: must be an element tag as a 2-tuple or int, or an element keyword
  warnings.warn(msg)
/home/rand/github/LoxLM/LoxLM/src/.venv/lib/python3.10/site-packages/pydicom/dataset.py:594: UserWarning: Invalid value 'ManufacturersModelName' used with the 'in' operator: must be an element tag as a 2-tuple or int, or an element keyword
  warnings.warn(msg)


In [34]:

cleaned_dicoms = clean_dicoms(dicoms)
cleaned_dicoms.to_json('input_dicoms.json', orient='records')

['root', 'SeriesDescription', 'TaskName', 'ProtocolName', 'RepetitionTime', 'EchoTime', 'InversionTime', 'PulseSequenceType', 'FlipAngle', 'Manufacturer', 'ManufacturersModelName']


In [35]:
df = pd.read_json("input_dicoms.json")

In [36]:
df

,file,root,SeriesDescription,TaskName,ProtocolName,RepetitionTime,EchoTime,InversionTime,PulseSequenceType,FlipAngle,Manufacturer,ManufacturersModelName
0,anat_t1w_mp_rage_1mm_pure-00038.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,Anat T1w MP-RAGE 1mm + PURE,NA,Chai,7.66,3.476,900.0,NA,7.0,GE MEDICAL SYSTEMS,NA
1,asset_calibration-00088.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,ASSET calibration,NA,Chai,1.372,0.5,NA,NA,0.0,GE MEDICAL SYSTEMS,NA
2,3_plane_loc-00013.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,3-Plane-Loc,NA,Chai,4.904,1.34,NA,NA,30.0,GE MEDICAL SYSTEMS,NA
3,slssfp_hres-00533.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,slssfp_hres,NA,Chai,3.068,1.436,NA,NA,35.0,GE MEDICAL SYSTEMS,NA
4,screen_save-00003.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/s...,Screen Save,NA,Chai,NA,NA,NA,NA,NA,GE MEDICAL SYSTEMS,NA
5,asset_calibration-00088.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,ASSET calibration,NA,Chai,1.372,0.5,NA,NA,0.0,GE MEDICAL SYSTEMS,NA
6,screen_save-00003.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/s...,Screen Save,NA,Chai,NA,NA,NA,NA,NA,GE MEDICAL SYSTEMS,NA
7,slssfp_hres-00533.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,slssfp_hres,NA,Chai,3.088,1.444,NA,NA,35.0,GE MEDICAL SYSTEMS,NA
8,slssfp_hres-00533.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,slssfp_hres,NA,Chai,3.068,1.436,NA,NA,35.0,GE MEDICAL SYSTEMS,NA
9,3_plane_loc-00013.dcm,/home/rand/github/LoxLM/LoxLM/20170828-25444/m...,3-Plane-Loc,NA,Chai,4.904,1.34,NA,NA,30.0,GE MEDICAL SYSTEMS,NA
